In [95]:
# Import dependencies
# Standard python libraries
import os
# Third-party libraries
from flask import Flask, redirect, request, url_for, render_template, session
import pandas as pd

# Internal imports
from db import get_df
from mfl import get_mfl_league, get_mfl_liveScoring, get_mfl_projectedScores

# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

user_league = "53906"
week = "2"
matchNumber = 0

In [96]:
# Get required data
leagueDF = get_mfl_league(user_league)
liveDF = get_mfl_liveScoring(user_league)
projDF = get_mfl_projectedScores(user_league, week)
players = get_df("player_df")
players = players.rename(columns={"PlayerID":"id_mfl"})

In [97]:
# Merge datasets
df = players.merge(
    projDF, on="id_mfl", how="left"
).merge(
    liveDF, on="id_mfl", how="left"
).merge(
    leagueDF, on="franchiseID", how="left"
)

In [98]:
# Reorder columns
df = df[["matchup", "franchiseName", "status", "Name", "Position", "Team", "sharkProjection", "liveScore", "secondsRemaining"]]

In [99]:
# Clean df
df.dropna(inplace=True)
# Define data types
df['sharkProjection'] = df['sharkProjection'].astype('float32', copy=False)
df['liveScore'] = df['liveScore'].astype('float32', copy=False)
df['secondsRemaining'] = df['secondsRemaining'].astype('int', copy=False)
df['matchup'] = df['matchup'].astype('int', copy=False)


In [100]:
# Calculate final Projections based on amount of time remaining
def calcScore(row):
    # Use a different calculation method for defenses since defenses do not accrue points
    if row["Position"] != "DF":
        result = row['liveScore'] + (row['sharkProjection'] * row['secondsRemaining'] / 3600)
    else:
        result = (row['sharkProjection'] * row['secondsRemaining'] + row['liveScore'] * (3600 - row['secondsRemaining'])) / 3600
    return result
df['finalProjection'] = df.apply(calcScore, axis=1)
df

,matchup,franchiseName,status,Name,Position,Team,sharkProjection,liveScore,secondsRemaining,finalProjection
0,4,O'Henry's Stiffies,starter,"Taylor, Jonathan",RB,IND,22.400000,7.5,0,7.500000
1,5,Comeback Kings,starter,"Allen, Josh",QB,BUF,24.200001,0.0,3600,24.200001
2,5,IDK much about soccer,starter,"Ekeler, Austin",RB,LAC,16.799999,13.1,0,13.100000
3,0,Saskatoon Squatches,starter,"Mahomes, Patrick",QB,KCC,23.600000,16.6,0,16.600000
4,1,Washington Fantasy Football Team,starter,"Kupp, Cooper",WR,LAR,16.299999,0.0,2036,9.218555
...,...,...,...,...,...,...,...,...,...,...
556,3,Gus The Bus,nonstarter,"Pierce, Dameon",RB,HOU,4.100000,2.0,2983,5.397305
652,0,Saskatoon Squatches,nonstarter,"Warren, Jaylen",RB,PIT,1.700000,2.2,0,2.200000
714,0,Crocodilopolis Body Snatchers,nonstarter,"Williams, Avery",RB,ATL,1.100000,0.0,2036,0.622111
948,4,O'Henry's Stiffies,starter,"Dortch, Greg",WR,ARI,1.100000,0.0,2658,0.812167


In [101]:
# Create matchup summary table
starts = df.loc[df['status']=="starter"]
matchSumm = starts.groupby(['matchup',"franchiseName"])['finalProjection'].sum()
matchSumm = pd.DataFrame(matchSumm)
matchSumm = matchSumm.reset_index()
matchSumm.loc[matchSumm.index%2==0, 'pivotIndex'] = "A" 
matchSumm.loc[matchSumm.index%2==1, 'pivotIndex'] = "B"
matchSumm = matchSumm[['matchup', 'pivotIndex', 'franchiseName', 'finalProjection']]
#matchSumm = matchSumm.set_index('matchup')
matchSumm = matchSumm.reset_index(drop=True)
matchSumm

,matchup,pivotIndex,franchiseName,finalProjection
0,0,A,Crocodilopolis Body Snatchers,190.329750
1,0,B,Saskatoon Squatches,212.904584
2,1,A,Pretty Big Wieners,144.575864
3,1,B,Washington Fantasy Football Team,122.973361
4,2,A,Down with the Sickness unto Death,191.286306
5,2,B,Verdansk Vaqueros FFC,201.286471
6,3,A,Gus The Bus,207.651056
7,3,B,Providence Monastic Spirits,148.322471
8,4,A,O'Henry's Stiffies,131.523779
9,4,B,WHAM! BAM! Thank You CAM!,134.456751


In [102]:
# M0FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M0PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']

In [103]:
# matchhead = pd.DataFrame()
# for matchNo in matchSumm.matchup.unique():
#     dfM = matchSumm.loc[matchSumm.matchup==matchNo]
#     dfM.columns = ['matchup', 'franchiseName', 'finalProjection']
#     matchhead = pd.concat([matchhead, dfM], axis=1)
# matchhead.drop(columns=['matchup', 'pivotIndex'], inplace=True)


In [104]:
M1A = matchSumm.loc[matchSumm.matchup==0].drop(columns=['matchup', 'pivotIndex'])
M2A = matchSumm.loc[matchSumm.matchup==1].drop(columns=['matchup', 'pivotIndex'])
M3A = matchSumm.loc[matchSumm.matchup==2].drop(columns=['matchup', 'pivotIndex'])
M4A = matchSumm.loc[matchSumm.matchup==3].drop(columns=['matchup', 'pivotIndex'])
M5A = matchSumm.loc[matchSumm.matchup==4].drop(columns=['matchup', 'pivotIndex'])
M6A = matchSumm.loc[matchSumm.matchup==5].drop(columns=['matchup', 'pivotIndex'])

In [105]:
M1A

,franchiseName,finalProjection
0,Crocodilopolis Body Snatchers,190.329750
1,Saskatoon Squatches,212.904584


In [115]:
#Get franchise Names based on which matchup is selected
franchiseA = list(matchSumm.loc[(matchSumm.matchup==matchNumber) & (matchSumm.pivotIndex=="A"), 'franchiseName'])[0]
franchiseB = list(matchSumm.loc[(matchSumm.matchup==matchNumber) & (matchSumm.pivotIndex=="B"), 'franchiseName'])[0]
franchiseA

'Crocodilopolis Body Snatchers'

In [116]:
# Select players on franchise roster
tableA = df.loc[df.franchiseName==franchiseA]
tableA.reset_index(inplace=True, drop=True)
# Create summary row
rowHead = ["summary", franchiseA, "Total", "", "", ""]
rowSummary = rowHead + list(tableA.loc[tableA.status=='starter'].sum()[['sharkProjection', 'liveScore', 'secondsRemaining', 'finalProjection']])
tableA.loc[len(tableA)] = rowSummary
# Create categories to sort by
tableA.status = pd.Categorical(tableA.status, 
                      categories=["nonstarter", 'Total', "starter"],
                      ordered=True)
tableA.Position = pd.Categorical(tableA.Position, 
        categories=['Def', 'PK', 'TE', 'WR', 'RB', 'QB', ""],
        ordered=True)
# Sort
tableA.sort_values(['status', 'Position', 'finalProjection'], inplace=True, ascending=False)
tableA.reset_index(inplace=True, drop=True)
# Rename columns
tableA.rename(columns={})
tableA

/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

,matchup,franchiseName,status,Name,Position,Team,sharkProjection,liveScore,secondsRemaining,finalProjection
0,0,Crocodilopolis Body Snatchers,starter,"Hurts, Jalen",QB,PHI,20.600000,0.000000,3600,20.600000
1,0,Crocodilopolis Body Snatchers,starter,"Murray, Kyler",QB,ARI,21.799999,-0.100000,2658,15.995666
2,0,Crocodilopolis Body Snatchers,starter,"Jacobs, Josh",RB,LVR,14.100000,6.100000,2658,16.510500
3,0,Crocodilopolis Body Snatchers,starter,"Mixon, Joe",RB,CIN,17.299999,1.800000,2717,14.856694
4,0,Crocodilopolis Body Snatchers,starter,"Montgomery, David",RB,CHI,13.300000,0.000000,3600,13.300000
5,0,Crocodilopolis Body Snatchers,starter,"Duvernay, Devin",WR,BAL,8.300000,21.500000,0,21.500000
6,0,Crocodilopolis Body Snatchers,starter,"Moore, D.J.",WR,CAR,9.400000,11.100000,0,11.100000
7,0,Crocodilopolis Body Snatchers,starter,"Brown, Marquise",WR,ARI,10.300000,2.100000,2658,9.704833
8,0,Crocodilopolis Body Snatchers,starter,"McKenzie, Isaiah",WR,BUF,8.800000,0.000000,3600,8.800000
9,0,Crocodilopolis Body Snatchers,starter,"Gesicki, Mike",TE,MIA,6.100000,11.100000,0,11.100000


In [117]:
# Select players on franchise roster
tableB = df.loc[df.franchiseName==franchiseB]
tableB.reset_index(inplace=True, drop=True)
# Create summary row
rowHead = ["summary", franchiseA, "Total", "", "", ""]
rowSummary = rowHead + list(tableB.loc[tableB.status=='starter'].sum()[['sharkProjection', 'liveScore', 'secondsRemaining', 'finalProjection']])
tableB.loc[len(tableB)] = rowSummary
# Create categories to sort by
tableB.status = pd.Categorical(tableB.status, 
                      categories=["nonstarter", 'Total', "starter"],
                      ordered=True)
tableB.Position = pd.Categorical(tableB.Position, 
        categories=['Def', 'PK', 'TE', 'WR', 'RB', 'QB', ""],
        ordered=True)
# Sort
tableB.sort_values(['status', 'Position', 'finalProjection'], inplace=True, ascending=False)
tableB.reset_index(inplace=True, drop=True)
# Rename columns
tableB.rename(columns={})
tableB

/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

,matchup,franchiseName,status,Name,Position,Team,sharkProjection,liveScore,secondsRemaining,finalProjection
0,0,Saskatoon Squatches,starter,"Carr, Derek",QB,LVR,17.900000,7.400000,2658,20.616166
1,0,Saskatoon Squatches,starter,"Mahomes, Patrick",QB,KCC,23.600000,16.600000,0,16.600000
2,0,Saskatoon Squatches,starter,"Williams, Javonte",RB,DEN,14.400000,1.600000,2983,13.532000
3,0,Saskatoon Squatches,starter,"Barkley, Saquon",RB,NYG,17.100000,11.700000,0,11.700000
4,0,Saskatoon Squatches,starter,"Stevenson, Rhamondre",RB,NEP,11.700000,6.300000,0,6.300000
5,0,Saskatoon Squatches,starter,"Hill, Tyreek",WR,MIA,12.800000,33.700001,0,33.700001
6,0,Saskatoon Squatches,starter,"Bateman, Rashod",WR,BAL,8.300000,17.799999,0,17.799999
7,0,Saskatoon Squatches,starter,"Chase, Ja'Marr",WR,CIN,15.100000,2.500000,2717,13.896306
8,0,Saskatoon Squatches,starter,"Jeudy, Jerry",WR,DEN,10.400000,1.600000,2983,10.217555
9,0,Saskatoon Squatches,starter,"Goedert, Dallas",TE,PHI,8.300000,0.000000,3600,8.300000
